In [13]:
import time
import numpy as np
from math import log,floor
from scipy.stats import norm
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

#Tool functions for FHE
from utils import draw_from_binary,draw_from_integer,draw_from_normal
from utils import mod, base_decomp, base_decomp_vec, draw_from_integer_vec

from fhe import LWE, inv_LWE
from fhe import RLWE,inv_RLWE,RLWE_prod
from fhe import RGSW,prod_ext

from utils import draw_from_binary_vec
from bootstrap import get_BK, accu_AP, extract_RLWE, key_switch
from bootstrap import get_BK_dec, accu_AP_dec



# Test Encryption and Decryption

In [14]:
# security parameters
N = int(2**8)
q = int(2**8)
sigma = 0.01
t = 2
num = 10
right = 0
for i in range(num):
    s = draw_from_binary(n=N)
    m_test = draw_from_binary(n=int(2**6))

    # perform encryption
    a_cifer,b_cifer = RLWE(n=N,q=q,sigma=sigma,s=s,t=t,m=m_test)
    # perform decryption
    m_test_dec = inv_RLWE(n=N,q=q,s=s,t=t,a=a_cifer,b=b_cifer)
    if m_test_dec == m_test:
        right += 1
print("The decryption is correct in ",right/num*100,"% of the cases")
        


The decryption is correct in  100.0 % of the cases


# Test RLWE_prod to perform product in homomorphically way

In [15]:
# security parameters
N = int(2**9)
q = int(2**8)
B = int(2**2)

print('k=',floor(log(q,B)))
sigma = 0.2
t = 2
s = draw_from_binary(n=N)

m0 = draw_from_binary(n=int(2**6))
m1 = np.poly1d([1,0,0])
# m1 = draw_from_binary(n=int(2**2))

# compute prod for clear messages
m0m1 = mod(poly=m0*m1,q=q,poly_modulus=np.poly1d([1] + ((N - 1) * [0]) + [1]))
print("m0*m1=",m0m1.coef)

# encode product
a_cifer,b_cifer = RLWE_prod(n=N,q=q,sigma=sigma,s=s,t=t,B=B,m0=m0,m1=m1)
# decode product
m0m1_decoded = inv_RLWE(n=N, q=q, s=s, t=t, a=a_cifer, b=b_cifer)
print("m0*m1_decoded=",m0m1_decoded.coef)
# Test equality
print("Is product recovered ?", m0m1==m0m1_decoded)

k= 4
m0*m1= [1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 0
 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0]
m0*m1_decoded= [1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 0 1 1 1 0
 0 1 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0]
Is product recovered ? True


# Test KeySwitch

In [16]:
# security parameters
N = int(2**9)
q = int(2**8)
B = int(2**2)

sigma = 0.5
t = 2

s_ini = draw_from_binary(n=N)
s_new = draw_from_binary(n=N)

print("if s_ini==s_new:",s_ini==s_new)
# draw message
m0 = draw_from_binary(n=int(2**3))
print("m0=",m0)

# code with secret key s
a_in,b_in = RLWE(n=N,q=q,sigma=sigma,s=s_ini,t=t,m=m0)
# Perform keyswitch
a_new, b_new = key_switch(n=N, q=q, B=B,sigma=0.1, t=t,s=s_new, s_prime=s_ini, a_prime=a_in, b_prime=b_in)
# decode
m_decoded = inv_RLWE(n=N, q=q, s=s_new, t=t, a=a_new, b=b_new)
# print("m_decoded=",m_decoded)

if s_ini==s_new: False
m0=    8     5     4
1 x + 1 x + 1 x + 1
powers= [ 1  4 16 64]
a_prime_dec= [poly1d([1, 0, 1, 0, 3, 0, 2, 2, 0, 3, 0, 1, 2, 1, 0, 2, 3, 0, 3, 0, 2, 1,
        1, 1, 3, 1, 3, 2, 0, 3, 0, 0, 2, 2, 2, 0, 0, 0, 3, 3, 0, 3, 2, 2,
        3, 3, 1, 3, 0, 3, 0, 2, 3, 1, 1, 2, 1, 3, 2, 0, 2, 3, 0, 3, 0, 1,
        0, 0, 0, 1, 2, 0, 2, 1, 3, 3, 1, 3, 1, 0, 0, 0, 2, 3, 1, 3, 3, 3,
        3, 2, 2, 3, 1, 1, 1, 3, 3, 0, 2, 0, 1, 1, 1, 0, 1, 1, 2, 0, 1, 3,
        1, 2, 0, 3, 0, 3, 3, 1, 3, 0, 0, 2, 2, 1, 3, 2, 0, 0, 3, 0, 3, 1,
        2, 2, 3, 1, 1, 1, 0, 3, 0, 1, 1, 2, 0, 0, 1, 2, 0, 2, 1, 2, 1, 0,
        2, 3, 1, 2, 3, 1, 1, 1, 0, 2, 3, 2, 1, 2, 2, 3, 1, 1, 3, 1, 3, 2,
        0, 1, 3, 3, 3, 2, 1, 2, 0, 2, 3, 0, 0, 1, 1, 2, 2, 3, 3, 3, 0, 0,
        3, 3, 3, 0, 2, 0, 2, 3, 2, 3, 1, 2, 1, 3, 0, 0, 2, 2, 1, 3, 0, 0,
        2, 2, 0, 2, 3, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1, 2, 0, 2, 0, 2, 0,
        1, 2, 1, 1, 3, 3, 0, 0, 1, 0, 2, 1, 1, 2, 2, 0, 1, 0, 1, 3, 3, 1,
        3, 1

TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:

np.random.seed(1234)
# security parameters
N = int(2**5)
q = int(2**6)
B = int(2**2)

print('k=',int(round(log(q,B))))
sigma = 0
t = 4

s_ini = np.zeros(N)
s_ini[0] = 1
s_ini = s_ini.astype(int)

s_new = draw_from_binary(n=N-1)

# get matrix of Bootstrap Keys
BK = get_BK(n=N,q=q,sigma=0,t=t,B=B,s=s_ini,s_prime=s_new)
print('BK:', BK)


In [ ]:

val = int(q/4)
coef_W = np.zeros(N)
coef_W[int(N/4):int(3*N/4)] = val
W = np.poly1d(coef_W.astype(int))
print('W:', W.coef)


In [ ]:

# draw message
m0 = np.array([0])
m1 = np.array([1])
print('m0:', m0)

p = 0
for _ in range(1): 
    a,b = LWE(n=N, q=q, sigma=0, s=s_ini, t=t, m=m0)
    # Step 1. ACC-part to get RLWE under s_prime
    a_prime, b_prime = accu_AP(n=N,q=q,B=B,a=a,b=b,w=W,BK=BK)
    #print('b_prime =',b_prime.coef)
    #print('coeff = ', inv_RLWE(n=N, q=q, s=s_new, t=t, a=a_prime, b=b_prime).coef)
    result = inv_RLWE(n=N, q=q, s=s_new, t=t, a=a_prime, b=b_prime).coef[-1]
    temp = mod(poly=b_prime-(a_prime*s_new),q=q,
               poly_modulus=np.poly1d([1] + (int(N - 1) * [0]) + [1]))
    print('temp', temp.coef)
    print('result =', result)
    p += result

In [ ]:
b = 16
monomial = np.poly1d([-1] + (N  - b) * [0])
print(monomial.coef)
mono = mod(poly=monomial*W,q=q,
           poly_modulus=np.poly1d([1] + ((N - 1) * [0]) + [1]))

In [ ]:
acc_a = np.poly1d([0])
acc_b = mod(poly=(monomial*W),q=q,
             poly_modulus=np.poly1d([1] + ((N - 1) * [0]) + [1]))

In [ ]:
print('coeff = ', inv_RLWE(n=N, q=q, s=s_new, t=t, a=acc_a, b=acc_b).coef)
result = inv_RLWE(n=N, q=q, s=s_new, t=t, a=acc_a, b=acc_b).coef[-1]

In [ ]:
n = N
# perform trivial RLWE(0,X^{-b} w(X))
acc_a = np.poly1d([0])
monomial = np.poly1d([-1] + (n - b) * [0])
acc_b = mod(poly=(monomial*W),q=q,
             poly_modulus=np.poly1d([1] + ((n - 1) * [0]) + [1]))
# perform recursive blind rotations
for i in range(n):
    # perform external prod RLWE x RGSW
    acc_a, acc_b = prod_ext(n=n,q=q,a=acc_a,b=acc_b,
                            B=B,M=BK[i,a[i]])

In [ ]:
p

In [ ]:
# Step 3. Extract cipher LWE of coeff_0(RLWE_{s})
a_0, b_0 = extract_RLWE(n=N,a=a_prime,b=b_prime,i_coeff=0)
if len(s_new.coef)<N:
    s_new_vec = np.insert(s_new.coef,0,(N-len(s_new.coef))*[0])
else:
    s_new_vec = s_new.coef
# decode message under key s_new
res = np.array([inv_LWE(n=N, q=q, s=s_new_vec, t=t, a=a_0, b=b_0) % 2])
print(res)